In [1]:
import Modelling_tools as MT
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

## Load the fully assembled dataframe

In [2]:
dataset = pd.read_csv("/Users/rmartinshort/Documents/Insight/Project/census_block_prediction/Fully_merged_dataset_Autogenerated_plus.csv")

In [3]:
dataset.columns

Index(['GISYEARJOIN', 'IDyear_x', 'LAT', 'LON', 'AREA', 'Nviolations',
       'Was complaint', 'Inspection Number', 'CrimeIsArson', 'CrimeIsOther',
       'Structure_fire', 'Vehicle_fire', 'External_fire',
       'Year Property Built', 'Assessed Land Value', 'Number of Units',
       'Lot Area', 'COMH', 'COMM', 'COMO', 'COMR', 'GOVT', 'IND', 'MISC',
       'MRES', 'SRES', 'UnkownUseType', 'A', 'B', 'C', 'D', 'S', 'H7X001',
       'H7X002', 'H7X003', 'H7X004', 'H7X005', 'H7X006', 'H7X007', 'H7X008',
       'H8C001', 'H8C002', 'H8C003', 'H8C004', 'H8C005', 'H8C006', 'H8C007',
       'H8C008', 'H8C009', 'IFC001', 'IFF001', 'IFF002', 'IFF003', 'IFF004',
       'Urban', 'IDyear_x.1', 'SF_pred', 'VF_pred', 'EF_pred', 'minyr',
       'maxyr', 'sdyr', 'resunits'],
      dtype='object')

In [4]:
dataset.head()

,GISYEARJOIN,IDyear_x,LAT,LON,AREA,Nviolations,Was complaint,Inspection Number,CrimeIsArson,CrimeIsOther,...,IFF004,Urban,IDyear_x.1,SF_pred,VF_pred,EF_pred,minyr,maxyr,sdyr,resunits
0,G060075002130020022007,2007,37.750066,-122.441075,16958.0,0.0,0.0,0.0,0.0,2.0,...,20,1,2007,0.0,0.0,0.0,1895.0,1989.0,25.519982,48.0
1,G060075002130020022008,2008,37.750066,-122.441075,16958.0,0.0,0.0,0.0,0.0,2.0,...,20,1,2008,0.0,0.0,0.0,1895.0,1989.0,25.519982,48.0
2,G060075002130020022009,2009,37.750066,-122.441075,16958.0,0.0,0.0,0.0,0.0,8.0,...,20,1,2009,0.0,0.0,0.0,1895.0,1989.0,25.519982,48.0
3,G060075002130020022010,2010,37.750066,-122.441075,16958.0,0.0,0.0,0.0,1.0,6.0,...,20,1,2010,0.0,0.0,0.0,1895.0,1989.0,25.519982,48.0
4,G060075002130020022011,2011,37.750066,-122.441075,16958.0,0.0,0.0,0.0,0.0,5.0,...,20,1,2011,0.0,0.0,0.0,1895.0,1989.0,25.519982,48.0


In [ ]:
#Do something about the NaNs
dataset['UnkownUseType'].fillna(1,inplace=True)
dataset['S'].fillna(1,inplace=True)
values = {'COMH':0, 'COMM':0, 'COMO':0, 'COMR':0, 'GOVT':0, 'IND':0, 'MISC':0,
       'MRES':0, 'SRES':0, 'A':0, 'B':0, 'C':0, 'D':0}
dataset = dataset.fillna(value=values)
dataset.fillna(dataset.mean(),inplace=True)

In [ ]:
dataset.drop(['IDyear_x.1'],inplace=True,axis=1)

In [ ]:
#This is a datasat for which we will predict the 2019 fires
use_for_future = dataset[dataset['IDyear_x']==2018]
#This is the dataset that we'll use to predict the 2018 fires
holdout_2018 = dataset[dataset['IDyear_x']==2017]
#This is the dataset that we'll train and test on
traintest = dataset[dataset['IDyear_x']<2017]

In [ ]:
firecols = ['SF_pred','VF_pred','EF_pred']
todrop = ['GISYEARJOIN']

In [ ]:
fires = traintest[firecols] #This is what we're trying to predict

#This is the dataset of predictors we're using to do the prediction
X = traintest.drop(firecols,axis=1)
X.drop(todrop,inplace=True,axis=1)

In [ ]:
X.isna().sum()

IDyear_x               0
LAT                    0
LON                    0
AREA                   0
Nviolations            0
Was complaint          0
Inspection Number      0
CrimeIsArson           0
CrimeIsOther           0
Structure_fire         0
Vehicle_fire           0
External_fire          0
Year Property Built    0
Assessed Land Value    0
Number of Units        0
Lot Area               0
COMH                   0
COMM                   0
COMO                   0
COMR                   0
GOVT                   0
IND                    0
MISC                   0
MRES                   0
SRES                   0
UnkownUseType          0
A                      0
B                      0
C                      0
D                      0
S                      0
H7X001                 0
H7X002                 0
H7X003                 0
H7X004                 0
H7X005                 0
H7X006                 0
H7X007                 0
H7X008                 0
H8C001                 0


In [ ]:
#Generate targert vector (1 or 0 depending on fire)
fire_to_predict = 'SF_pred'
y = fires[fire_to_predict].apply(MT.classify)

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
test_parameters = {
        'classify__n_estimators': (10,20,30),
        'classify__max_depth': (5,8,10),
        'classify__min_samples_leaf':(50,60,80),
}


#Parameters to search over for gradient boosting
test_parameters_GB = {
    
    'classify__n_estimators': (10,20,30),
    'classify__n_learning_rate': (0.5,1,2),
    'classify__max_depth': (5,8,10),
    'classify__min_samples_leaf': (50,60,100),
}

#Parameters to search over for Logistic regression
test_parameters_LRC = {
    
    'classify__C': (0.1,1,10),
    'classify__penalty': ('L1','L2'),
    'classify__class_weight': ('balanced',None),
}

In [ ]:
pipeline, gridsearch, testparams = MT.generate_pipeline_RC(test_parameters=test_parameters)

In [ ]:
#Run grid search for the best model
Xcols, estimator = MT.perform_grid_search(pipeline,gridsearch,X,y,testparams,list(X.columns),select=False)

Performing grid search...
pipeline: ['scale', 'select', 'classify']
parameters:
{'classify__n_estimators': (10, 20, 30), 'classify__max_depth': (5, 8, 10), 'classify__min_samples_leaf': (50, 60, 80)}
Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   49.5s


In [ ]:
firesholdout = holdout_2018[firecols] #This is what we're trying to predict
yholdout = firesholdout[fire_to_predict].apply(MT.classify)
Xholdout = holdout_2018.drop(firecols,axis=1) #This is what we're using to predict
Xholdout.drop(todrop,inplace=True,axis=1)

In [ ]:
Xholdout.columns

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import learning_curve
import numpy as np
def plot_ROC_curve(Xdata,Ytrue,model,title):

    '''
    Plot ROC curve for this model 
    '''

    fires_holdout_predict = model.predict_proba(Xdata)

    fpr, tpr, thresholds1 = roc_curve(np.array(Ytrue).astype(int), fires_holdout_predict[:,1])


    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    ax.plot(fpr,tpr)
    ax.set_xlabel("False Positive")
    ax.set_ylabel("True Positive")
    ax.plot(np.linspace(0,1,10),np.linspace(0,1,10),'k--')
    ax.set_title('%s' %title)
    fname = 'AOC_%s.png' %title
    plt.savefig(fname,dpi=400)

    return fig

In [ ]:
def plot_Learning_curve(estimator, title, X, y, ylim=None, cv=None,
                  n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5),score_type='roc_auc'):

    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    fig = plt.figure(figsize=(10,10))
    ax=fig.add_subplot(111)
    ax.set_title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    ax.set_xlabel("Training examples")
    ax.set_ylabel("ROC AUC Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes,scoring=score_type)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    ax.grid()

    ax.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    ax.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    ax.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    ax.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    ax.legend(loc="best")

    fname = 'LC_%s.png' %title
    plt.savefig(fname,dpi=400)


    return fig

In [ ]:
ROCplot = plot_ROC_curve(Xholdout,yholdout,estimator,title='SF_ROC_curve')

In [ ]:
LC = plot_Learning_curve(estimator,'SF_test',X,y,cv=5,n_jobs=4,train_sizes=np.linspace(.01, 1.0, 5))

## What about if we don't do feature selection?

In [26]:
pipeline, gridsearch, testparams = MT.generate_pipeline_RC(test_parameters=test_parameters,select=False)

In [27]:
#Run grid search for the best model
Xcols, estimator = MT.perform_grid_search(pipeline,gridsearch,X,y,testparams,list(X.columns),select=False)

TypeError: perform_grid_search() got an unexpected keyword argument 'select'

In [ ]:
firesholdout = holdout_2018[firecols] #This is what we're trying to predict
yholdout = firesholdout[fire_to_predict].apply(MT.classify)
Xholdout = holdout_2018.drop(firecols,axis=1) #This is what we're using to predict
Xholdout.drop(todrop,inplace=True,axis=1)

In [ ]:
ROCplot = plot_ROC_curve(Xholdout,yholdout,estimator,title='SF_ROC_curve')